In [3]:
import numpy as np, jupyter, matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robbypeery/college-basketball-pbp-23-24")

print("Path to dataset files:", path)

Y:\PycharmProjects\ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Zak\.cache\kagglehub\datasets\robbypeery\college-basketball-pbp-23-24\versions\18


In [5]:
df = pd.read_csv((('./CBBD/Colorado_pbp.csv')))

In [6]:
print(df.shape)
df.head(6)
# df.score_diff
# df["game_id"].unique().shape

(10858, 34)


,game_id,date,home,away,play_id,half,time_remaining_half,secs_remaining,secs_remaining_absolute,description,...,arena_location,arena,attendance,shot_team,shot_outcome,shooter,three_pt,free_throw,possession_before,possession_after
0,401587141,2023-11-06,Colorado,Towson,1,1,19:42,2382,2382,KJ Simpson made Layup. Assisted by J'Vonne Had...,...,"Boulder, CO",CU Events Center,6078,Colorado,made,KJ Simpson,False,False,Colorado,Towson
1,401587141,2023-11-06,Colorado,Towson,2,1,19:20,2360,2360,Charles Thompson missed Jumper.,...,"Boulder, CO",CU Events Center,6078,Towson,missed,Charles Thompson,False,False,Towson,Colorado
2,401587141,2023-11-06,Colorado,Towson,3,1,19:17,2357,2357,KJ Simpson Defensive Rebound.,...,"Boulder, CO",CU Events Center,6078,NaN,NaN,NaN,NaN,NaN,Towson,Colorado
3,401587141,2023-11-06,Colorado,Towson,4,1,19:13,2353,2353,KJ Simpson missed Three Point Jumper.,...,"Boulder, CO",CU Events Center,6078,Colorado,missed,KJ Simpson,True,False,Colorado,Towson
4,401587141,2023-11-06,Colorado,Towson,5,1,19:10,2350,2350,Tyler Tejada Defensive Rebound.,...,"Boulder, CO",CU Events Center,6078,NaN,NaN,NaN,NaN,NaN,Colorado,Towson
5,401587141,2023-11-06,Colorado,Towson,6,1,18:52,2332,2332,Tyler Tejada Turnover.,...,"Boulder, CO",CU Events Center,6078,NaN,NaN,NaN,NaN,NaN,Towson,Colorado


In [7]:
df = df.sort_values(['game_id', 'play_id'], ascending=[True,True]).reset_index(drop=True)
runs=[]

for game_id, game_df in df.groupby('game_id'):
    home = game_df.iloc[0]['home']
    away = game_df.iloc[0]['away']
    
    # We track cumulative points scored
    home_score = 0
    away_score = 0
    
    # We also track potential runs
    run_points = {home: 0, away: 0}
    run_start_index = None
    run_start_time = None
    
    for idx, row in game_df.iterrows():
        new_home_score = row['home_score']
        new_away_score = row['away_score']
        
        # Determine who scored (if anyone)
        if new_home_score != home_score or new_away_score != away_score:
            # Someone scored
            scoring_team = home if new_home_score != home_score else away
            points_scored = (new_home_score - home_score) if scoring_team == home else (new_away_score - away_score)
            
            if run_points[scoring_team] == 0:
                # First points in the run
                run_start_index = idx
                run_start_time = row['time_remaining_half']
            
            run_points[scoring_team] += points_scored
            # Reset the opponent's run points if they score
            opponent = away if scoring_team == home else home
            run_points[opponent] = 0
            
            # Check if run is 6-0
            if run_points[scoring_team] >= 6 and run_points[opponent] == 0:
                runs.append({
                    'game_id': game_id,
                    'team': scoring_team,
                    'start_play_id': game_df.loc[run_start_index]['play_id'],
                    'start_time': run_start_time,
                    'end_play_id': row['play_id'],
                    'end_time': row['time_remaining_half'],
                    'points_scored': run_points[scoring_team]
                })
                
                # After finding a run, reset
                run_points[scoring_team] = 0
                run_start_index = None
                run_start_time = None
        
        home_score = new_home_score
        away_score = new_away_score

# Turn the runs list into a DataFrame
runs_df = pd.DataFrame(runs)

In [8]:
runs_df

,game_id,team,start_play_id,start_time,end_play_id,end_time,points_scored
0,401576684,Colorado,44,13:43,55,12:29,7
1,401576684,Colorado,87,8:34,95,8:02,6
2,401576684,Colorado,115,6:24,137,3:59,6
3,401576684,Colorado,166,19:54,169,19:34,6
4,401576684,Colorado,280,6:53,293,5:46,6
...,...,...,...,...,...,...,...
196,401625480,Colorado,257,4:54,266,3:48,7
197,401625482,Washington St,178,12:22,190,10:53,7
198,401625483,Colorado,41,13:34,50,12:28,6
199,401625483,Colorado,51,12:28,82,8:52,7


In [9]:
df[(df['game_id']==401576684) &
    (df['play_id'] >= 43) &
    (df['play_id'] <= 56)]

,game_id,date,home,away,play_id,half,time_remaining_half,secs_remaining,secs_remaining_absolute,description,...,arena_location,arena,attendance,shot_team,shot_outcome,shooter,three_pt,free_throw,possession_before,possession_after
42,401576684,2023-12-03,Colorado,Pepperdine,43,1,13:59,2039,2039,Cord Stansberry made Three Point Jumper. Assis...,...,"Boulder, CO",CU Events Center,7231,Pepperdine,made,Cord Stansberry,True,False,Pepperdine,Colorado
43,401576684,2023-12-03,Colorado,Pepperdine,44,1,13:43,2023,2023,Julian Hammond III made Three Point Jumper. As...,...,"Boulder, CO",CU Events Center,7231,Colorado,made,Julian Hammond III,True,False,Colorado,Pepperdine
44,401576684,2023-12-03,Colorado,Pepperdine,45,1,13:26,2006,2006,Cord Stansberry Turnover.,...,"Boulder, CO",CU Events Center,7231,NaN,NaN,NaN,NaN,NaN,Pepperdine,Colorado
45,401576684,2023-12-03,Colorado,Pepperdine,46,1,13:26,2006,2006,J'Vonne Hadley Steal.,...,"Boulder, CO",CU Events Center,7231,NaN,NaN,NaN,NaN,NaN,Pepperdine,Colorado
46,401576684,2023-12-03,Colorado,Pepperdine,47,1,13:22,2002,2002,Julian Hammond III made Layup. Assisted by J'V...,...,"Boulder, CO",CU Events Center,7231,Colorado,made,Julian Hammond III,False,False,Colorado,Pepperdine
47,401576684,2023-12-03,Colorado,Pepperdine,48,1,13:22,2002,2002,Pepperdine Timeout,...,"Boulder, CO",CU Events Center,7231,NaN,NaN,NaN,NaN,NaN,Colorado,Pepperdine
48,401576684,2023-12-03,Colorado,Pepperdine,49,1,13:01,1981,1981,Michael Ajayi missed Jumper.,...,"Boulder, CO",CU Events Center,7231,Pepperdine,missed,Michael Ajayi,False,False,Pepperdine,Colorado
49,401576684,2023-12-03,Colorado,Pepperdine,50,1,12:57,1977,1977,Julian Hammond III Defensive Rebound.,...,"Boulder, CO",CU Events Center,7231,NaN,NaN,NaN,NaN,NaN,Pepperdine,Colorado
50,401576684,2023-12-03,Colorado,Pepperdine,51,1,12:45,1965,1965,Eddie Lampkin Jr. missed Layup.,...,"Boulder, CO",CU Events Center,7231,Colorado,missed,Eddie Lampkin Jr.,False,False,Colorado,Pepperdine
51,401576684,2023-12-03,Colorado,Pepperdine,52,1,12:44,1964,1964,Pepperdine Defensive Rebound.,...,"Boulder, CO",CU Events Center,7231,NaN,NaN,NaN,NaN,NaN,Colorado,Pepperdine


In [10]:
from sklearn.metrics import precision_score

def time_to_decimal(t):
    minutes, seconds = map(int, t.split(':'))
    return minutes + seconds / 60

df['time_remaining_half_decimal'] = df['time_remaining_half'].apply(time_to_decimal)

def create_features(play):
    return pd.Series([
        1 if play['shot_outcome'] == 'made' else 0 if play['shot_outcome'] == 'missed' else -1,
        int(play['three_pt']) if pd.notna(play['three_pt']) else 0,
        int(play['free_throw']) if pd.notna(play['free_throw']) else 0,
        play['home_score'],
        play['away_score'],
        play['score_diff'],
        play['time_remaining_half_decimal'] / 20.0  # Normalize time (assuming 20 min half)
    ])

# Build feature matrix
play_features = df.apply(create_features, axis=1).values.tolist()

# --- Windowing ---
window_size = 3
lookahead_plays = 5

X = []
y = []

for i in range(len(play_features) - window_size - lookahead_plays):
    window = []
    for j in range(window_size):
        window.extend(play_features[i + j])
    
    # Lookahead
    home_score_start = df.iloc[i + window_size - 1]['home_score']
    away_score_start = df.iloc[i + window_size - 1]['away_score']
    
    home_score_end = df.iloc[i + window_size + lookahead_plays - 1]['home_score']
    away_score_end = df.iloc[i + window_size + lookahead_plays - 1]['away_score']
    
    home_diff = home_score_end - home_score_start
    away_diff = away_score_end - away_score_start
    
    # Did any team go on 6-0 run?
    if home_diff >= 6 and away_diff == 0:
        label = 1
    elif away_diff >= 6 and home_diff == 0:
        label = 1
    else:
        label = 0

    X.append(window)
    y.append(label)

X = np.array(X)
y = np.array(y)

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Model training ---
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# --- Evaluation ---
y_pred = model.predict(X_test)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2165
           1       0.00      0.00      0.00         5

    accuracy                           1.00      2170
   macro avg       0.50      0.50      0.50      2170
weighted avg       1.00      1.00      1.00      2170



Y:\PycharmProjects\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Y:\PycharmProjects\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Y:\PycharmProjects\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
